In [1]:
import mmengine
import mmcv
from mmcv.transforms import Compose, Resize
import matplotlib.pyplot as plt
from mmdet.utils import get_test_pipeline_cfg
import torch
import numpy as np
import torchvision.transforms as torch_trans
from mmdet.apis import init_detector, inference_detector 
from wrappers import *
from mmdet.visualization import DetLocalVisualizer
from mmengine.registry import MODELS
from mmengine.config import Config

In [2]:
model_path = "configs/swin/swin_tiny.py"
# model_path = "configs/swin_based/swin_tiny.py"
ckpt_path = "weights/swin_tiny_patch4_window7_224.pth"
cfg = Config.fromfile(model_path)

model = init_detector(cfg, ckpt_path)
# model = MODELS.build(cfg.model)

vis = DetLocalVisualizer()
palette = [(220, 20, 60), (119, 11, 32), (0, 0, 142), (0, 0, 230), (0, 200, 30), (100, 200, 150), (100, 200, 50), (10, 200, 50)]
CLASSES = ('1','2','3','4','5','6')
img = mmcv.imread("data/mmdet/NEU_DET/test/inclusion_243.jpg")

Loads checkpoint by local backend from path: weights/swin_tiny_patch4_window7_224.pth
The model and loaded state dict do not match exactly

unexpected key in source state_dict: model

missing keys in source state_dict: backbone.patch_embed.projection.weight, backbone.patch_embed.projection.bias, backbone.patch_embed.norm.weight, backbone.patch_embed.norm.bias, backbone.stages.0.blocks.0.norm1.weight, backbone.stages.0.blocks.0.norm1.bias, backbone.stages.0.blocks.0.attn.w_msa.relative_position_bias_table, backbone.stages.0.blocks.0.attn.w_msa.relative_position_index, backbone.stages.0.blocks.0.attn.w_msa.qkv.weight, backbone.stages.0.blocks.0.attn.w_msa.qkv.bias, backbone.stages.0.blocks.0.attn.w_msa.proj.weight, backbone.stages.0.blocks.0.attn.w_msa.proj.bias, backbone.stages.0.blocks.0.norm2.weight, backbone.stages.0.blocks.0.norm2.bias, backbone.stages.0.blocks.0.ffn.layers.0.0.weight, backbone.stages.0.blocks.0.ffn.layers.0.0.bias, backbone.stages.0.blocks.0.ffn.layers.1.weight, ba

/home/huemorgen/miniconda3/envs/mmlab/lib/python3.8/site-packages/mmdet/apis/inference.py:90: UserWarning: dataset_meta or class names are not saved in the checkpoint's meta data, use COCO classes by default.
  warnings.warn(


In [3]:
from mmengine.runner.checkpoint import save_checkpoint
file = 'swin_base_model.pth'
save_checkpoint(model.backbone.state_dict(), file)

06/01 16:42:30 - mmengine - WARNING - "FileClient" will be deprecated in future. Please use io functions in https://mmengine.readthedocs.io/en/latest/api/fileio.html#file-io
06/01 16:42:30 - mmengine - WARNING - "HardDiskBackend" is the alias of "LocalBackend" and the former will be deprecated in future.


In [4]:

model_path = "configs/swin/swin_moe.py"
cfg = Config.fromfile(model_path)
model = init_detector(cfg)
backbone = model.backbone

ModuleList(
  (0): SwinBlockSequence(
    (blocks): ModuleList(
      (0-1): 2 x SwinBlock(
        (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
        (attn): ShiftWindowMSA(
          (w_msa): WindowMSA(
            (qkv): Linear(in_features=96, out_features=288, bias=True)
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=96, out_features=96, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
            (softmax): Softmax(dim=-1)
          )
          (drop): DropPath()
        )
        (norm2): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
        (ffn): FFN(
          (layers): Sequential(
            (0): Sequential(
              (0): Linear(in_features=96, out_features=384, bias=True)
              (1): GELU(approximate='none')
              (2): Dropout(p=0.0, inplace=False)
            )
            (1): Linear(in_features=384, out_features=96, bias=True)
            (2): Dropout(p=0.0, inpl

/home/huemorgen/miniconda3/envs/mmlab/lib/python3.8/site-packages/mmdet/apis/inference.py:70: UserWarning: checkpoint is None, use COCO classes by default.
  warnings.warn('checkpoint is None, use COCO classes by default.')


In [5]:
from mmengine.runner.checkpoint import _load_checkpoint
ckpt_path = file
checkpoint = _load_checkpoint(ckpt_path, 'cpu')
ckp_state_dict = checkpoint
backbone_state_dict = backbone.state_dict()


Loads checkpoint by local backend from path: swin_base_model.pth


In [6]:
gates = []
for i, key in enumerate(backbone_state_dict.keys()):
    if 'gate' in key:
        gates.append(key)

In [7]:
gates

['stages.3.blocks.0.ffn.gate.w_gate.weight',
 'stages.3.blocks.0.ffn.gate.w_gate.bias']

In [8]:
experts = []
for i, key in enumerate(backbone_state_dict.keys()):
    if 'experts' in key:
        experts.append(key)

ffn_ckpt = []
for i, key in enumerate(ckp_state_dict.keys()):
    if 'ffn' in key:
        ffn_ckpt.append(key)

In [9]:
experts,ffn_ckpt

(['stages.3.blocks.0.ffn.experts.0.layers.0.0.weight',
  'stages.3.blocks.0.ffn.experts.0.layers.0.0.bias',
  'stages.3.blocks.0.ffn.experts.0.layers.1.weight',
  'stages.3.blocks.0.ffn.experts.0.layers.1.bias',
  'stages.3.blocks.0.ffn.experts.1.layers.0.0.weight',
  'stages.3.blocks.0.ffn.experts.1.layers.0.0.bias',
  'stages.3.blocks.0.ffn.experts.1.layers.1.weight',
  'stages.3.blocks.0.ffn.experts.1.layers.1.bias'],
 ['stages.0.blocks.0.ffn.layers.0.0.weight',
  'stages.0.blocks.0.ffn.layers.0.0.bias',
  'stages.0.blocks.0.ffn.layers.1.weight',
  'stages.0.blocks.0.ffn.layers.1.bias',
  'stages.0.blocks.1.ffn.layers.0.0.weight',
  'stages.0.blocks.1.ffn.layers.0.0.bias',
  'stages.0.blocks.1.ffn.layers.1.weight',
  'stages.0.blocks.1.ffn.layers.1.bias',
  'stages.1.blocks.0.ffn.layers.0.0.weight',
  'stages.1.blocks.0.ffn.layers.0.0.bias',
  'stages.1.blocks.0.ffn.layers.1.weight',
  'stages.1.blocks.0.ffn.layers.1.bias',
  'stages.1.blocks.1.ffn.layers.0.0.weight',
  'stages.1.bl

In [10]:
num_experts = 2
j = 1
ckp_it = iter(ckp_state_dict)
model_it = iter(backbone_state_dict)
experts_it = iter(experts)
l = len([key for key in ckp_state_dict])
it = 0
stop_it = False
while not stop_it:
    model_key = next(model_it)
    print("Module")
    print(model_key)
    if 'gate' in model_key:
        continue
   
    if 'expert' in model_key:
        ws = []
        for i in range(4):
            key = next(ckp_it)
            w = ckp_state_dict[key]
            ws.append(w)
            it+=1
            if it==l:
                stop_it = True
                break
        i = 0
        for _ in range(4*num_experts):
            shape = backbone_state_dict[model_key].shape
            # shape = [shape[0], shape[1]] if len(shape)==2 else [shape[0]]
            print(shape)
            print(ws[i%4].shape)
            backbone_state_dict[model_key] = ws[i%4].clone()
            i+=1
            model_key = next(model_it)  
        if 'gate' in model_key:
            model_key = next(model_it)
            continue
    ckp_key = next(ckp_it)
    print(ckp_key)
    it+=1
    if it==l:
            stop_it = True
            break
    backbone_state_dict[model_key] = ckp_state_dict[ckp_key].clone()
    


Module
patch_embed.projection.weight
patch_embed.projection.weight
Module
patch_embed.projection.bias
patch_embed.projection.bias
Module
patch_embed.norm.weight
patch_embed.norm.weight
Module
patch_embed.norm.bias
patch_embed.norm.bias
Module
stages.0.blocks.0.norm1.weight
stages.0.blocks.0.norm1.weight
Module
stages.0.blocks.0.norm1.bias
stages.0.blocks.0.norm1.bias
Module
stages.0.blocks.0.attn.w_msa.relative_position_bias_table
stages.0.blocks.0.attn.w_msa.relative_position_bias_table
Module
stages.0.blocks.0.attn.w_msa.relative_position_index
stages.0.blocks.0.attn.w_msa.relative_position_index
Module
stages.0.blocks.0.attn.w_msa.qkv.weight
stages.0.blocks.0.attn.w_msa.qkv.weight
Module
stages.0.blocks.0.attn.w_msa.qkv.bias
stages.0.blocks.0.attn.w_msa.qkv.bias
Module
stages.0.blocks.0.attn.w_msa.proj.weight
stages.0.blocks.0.attn.w_msa.proj.weight
Module
stages.0.blocks.0.attn.w_msa.proj.bias
stages.0.blocks.0.attn.w_msa.proj.bias
Module
stages.0.blocks.0.norm2.weight
stages.0.blo

In [11]:
from mmengine.runner.checkpoint import save_checkpoint
checkpoint_out = dict(state_dict=backbone_state_dict)
file = 'moe_base_model.pth'
save_checkpoint(checkpoint_out, file)